# Day 19

## Part 1

- to crack geodes we need geode-cracking obsidian robots
- to get obsidian we need robots water-proofed with clay
- to get clay we need clay-collecting robots
- all robots need ore, which is gathered by ore-collecitng robots (with big drills)
- I have one of those ore-collecing robots already!
- robots can collect 1 unit of resource per minute
- it takes 1 minute to build a robot but resourses are consumed the instant the robot construction begins
- we have to choose a blueprint (from the input) and stick with it for the run
- a blueprint's quality is its ID number times the number of geodes we can mine using it in `24 minutes`

` What do you get if you add up the quality level of all of the blueprints in your list?`

In [32]:
from __future__ import annotations

from copy import deepcopy
from dataclasses import dataclass
from utils import parse_from_file, ParseConfig

@dataclass
class Robot:
    ore: int

@dataclass
class OreRobot(Robot):
    pass

@dataclass
class ClayRobot(Robot):
    pass

@dataclass
class ObsidianRobot(Robot):
    clay: int

@dataclass
class GeodeRobot(Robot):
    obsidian: int

@dataclass
class Blueprint:
    id: int
    ore_bot: OreRobot
    clay_bot: ClayRobot
    obsidian_bot: ObsidianRobot
    geode_bot: GeodeRobot

parser = ParseConfig('\n', ParseConfig(': ', [
    ParseConfig(' ', [None, int]),
    ParseConfig('.', [
        ParseConfig(' ', [None]*4 + [int] + [None]),
        ParseConfig(' ', [None]*5 + [int] + [None]),
        ParseConfig(' ', [None]*5 + [int] + [None]*2 + [int] + [None]),
        ParseConfig(' ', [None]*5 + [int] + [None]*2 + [int] + [None]),
        None
    ])
]))

blueprint_values = parse_from_file(
        'test_inputs\\day_19.txt', parser, unnest_single_items=True)

blueprints = []
for id, (ore, clay, obsidian, geode) in blueprint_values:
    blueprints.append(Blueprint(
        id,
        OreRobot(ore),
        ClayRobot(clay),
        ObsidianRobot(*obsidian),
        GeodeRobot(*geode)
    ))

print(blueprints[0])

Blueprint(id=1, ore_bot=OreRobot(ore=4), clay_bot=ClayRobot(ore=2), obsidian_bot=ObsidianRobot(ore=3, clay=14), geode_bot=GeodeRobot(ore=2, obsidian=7))


In [58]:
@dataclass
class State:
    ore: int
    clay: int
    obsidian: int
    geodes: int
    ore_bots: int
    clay_bots: int
    obsidian_bots: int
    geode_bots: int

    @property
    def hashable(self) -> tuple[int]:
        return self.ore, self.clay, self.obsidian, self.geodes, \
            self.ore_bots, self.clay_bots, self.obsidian_bots, self.geode_bots
    
    def harvest(self) -> None:
        """
        updates the resource values
        """
        self.ore += self.ore_bots
        self.clay += self.clay_bots
        self.obsidian += self.obsidian_bots
        self.geodes += self.geode_bots

    def build(self, robot: Robot|None) -> State:
        """
        returns a copy of this state with the resources required to build
        that robot consumed
        """
        temp = State(*self.hashable)
        if robot is None:
            pass
        elif isinstance(robot, OreRobot):
            temp.ore -= robot.ore
        elif isinstance(robot, ClayRobot):
            temp.ore -= robot.ore
        elif isinstance(robot, ObsidianRobot):
            temp.ore -= robot.ore
            temp.clay -= robot.clay
        elif isinstance(robot, GeodeRobot):
            temp.ore -= robot.ore
            temp.obsidian -= robot.obsidian
        else:
            raise ValueError(f'robot type not recongised: {robot}')
        
        return temp

    def get_options(self, blueprint: Blueprint) -> list[State]:
        """
        returns a list of possible states for a given blueprint
        """
        options = []
        # build nothing
        temp = self.build(None)
        temp.harvest()
        options.append(temp)
        # build an ore bot
        robot = blueprint.ore_bot
        if self.ore >= robot.ore:
            temp = self.build(robot)
            temp.harvest()
            temp.ore_bots += 1
            options.append(temp)
        # build a clay bot
        robot = blueprint.clay_bot
        if self.ore >= robot.ore:
            temp = self.build(robot)
            temp.harvest()
            temp.clay_bots += 1
            options.append(temp)
        # build an obsidian bot
        robot = blueprint.obsidian_bot
        if self.ore >= robot.ore and self.clay >= robot.clay:
            temp = self.build(robot)
            temp.harvest()
            temp.obsidian_bots += 1
            options.append(temp)
        # bulid a geode bot
        robot = blueprint.geode_bot
        if self.ore >= robot.ore and self.obsidian >= robot.obsidian:
            temp = self.build(robot)
            temp.harvest()
            temp.geode_bots += 1
            options.append(temp)

        return options

    def max_geodes(self, time_remaining: int) -> int:
        """
        calculates the maximum possible number of geodes that could be
        harvested in the remaining time

        assumes that it is possible to gather enough resources to create a
        geode bot every minute until the end giving a linearly ramping total
        """
        geodes = self.geodes
        potential_bots = self.geode_bots
        for _ in range(time_remaining + 1):
            geodes += potential_bots
            potential_bots += 1
        return geodes

start = State(0, 0, 0, 0, 1, 0, 0, 0)

def get_layers(
    start: State, blueprint: Blueprint, total_time: int
) -> list[list[State]]:
    """
    returns a list of all possible states at each iteration
    """
    layers = [[start.hashable]]
    for iteration in range(1, total_time + 1):
        print(
            f'iteration: {iteration}, layer size: {len(layers[-1])}', end='\r')
        new_states = []
        # get all possible subsequent state options
        for values in layers[-1]:
            new_states.extend(State(*values).get_options(blueprint))
        # determine the maximum possible number of geodes from this layer
        max_geodes = max([
            state.max_geodes(total_time - iteration) for state in new_states])
        # add only those that could potentially maximise geodes and aren't
        # duplicates
        new_layer = []
        for state in new_states:
            if (
                state.hashable not in new_layer and
                state.max_geodes(total_time - iteration) == max_geodes
            ):
                new_layer.append(state.hashable)
        layers.append(new_layer)
    return [[State(*state) for state in layer] for layer in layers]

layers = get_layers(start, blueprints[0], 24)

print(layers[-1][:3])

[State(ore=1, clay=26, obsidian=3, geodes=9, ore_bots=1, clay_bots=4, obsidian_bots=3, geode_bots=3), State(ore=1, clay=27, obsidian=3, geodes=9, ore_bots=1, clay_bots=4, obsidian_bots=3, geode_bots=3)]
